<a href="https://colab.research.google.com/github/andresarpi/tf_for_dl_research/blob/master/lecture_1_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
a = tf.add(3, 5)
sess = tf.Session()
print(sess.run(a))
sess.close()

8


In [0]:
x = 2
y = 3
op1 =tf.add(x, y)
op2 = tf.multiply(x, y)
op3 = tf.pow(op2, op1)
with tf.Session() as sess:
  op3 = sess.run(op3)

In [0]:
x = 2
y = 3
add_op =tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op)
with tf.Session() as sess:
  z = sess.run(pow_op)

In [10]:
z, (2+3) ** (2*3)

(15625, 15625)

In [0]:
x = 2
y = 3
add_op =tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op)
with tf.Session() as sess:
  z, not_useless = sess.run([pow_op, useless])

In [12]:
not_useless, 2 * (2+ 3)

(10, 10)

You can break up graphs into chuncks and run them in parallel in different computation units.


In [14]:
# Creates a graph. Make sure you have at least one GPU to run this
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='b')
  c = tf.multiply(a, b)

# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Runs the op.
print(sess.run(c))


[ 1.  4.  9. 16. 25. 36.]
